In [2]:
import os
os.chdir('/content/drive/MyDrive/research/power system/power_system_DM5_803')

In [3]:
import numpy as np
from torch.distributions import Categorical, Normal
from memory import Memory
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam,SGD
from util import *
from copy import deepcopy
from torch.optim.lr_scheduler import ExponentialLR

In [4]:
def fanin_init(size, fanin=None):
    fanin = fanin or size[0]
    v = 1. / np.sqrt(fanin)
    return torch.Tensor(size).uniform_(-v, v)
class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions, hidden1=128, hidden2=128, init_w=3e-3):
        super(Actor, self).__init__()
        #self.constraints = constraints
        self.fc1 = nn.Linear(nb_states, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2,nb_actions) #production
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.init_weights(init_w)
    def init_weights(self, init_w):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-init_w, init_w)
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.tanh(out)#[-1,1]
        return out
class Critic(nn.Module):
    def __init__(self,nb_states,nb_actions,hidden1 = 128,hidden2 = 128,init_w = 3e-3):
        super(Critic, self).__init__()
       # self.constraints = constraints
        self.fc1 = nn.Linear(nb_states+nb_actions, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 1) 
        self.relu = nn.ReLU()
        self.init_weights(init_w)
    def init_weights(self, init_w):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.fc3.weight.data.uniform_(-init_w, init_w)
    def forward(self,state,action):
        x = torch.concat((state,action),dim = 1)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out
    def mc(self,state,action):
      state.requires_grad = False
      action.requires_grad = True
      self.forward(state,action).backward()
      return action.grad.item()


In [21]:
criterion = nn.MSELoss()
class DDPG(object):
    def __init__(self, nb_interval,nb_states,nb_actions, constraints, args):
        if args.seed > 0:
            self.seed(args.seed)
        self.constraints = constraints
        self.nb_interval = nb_interval
        self.nb_states = nb_states
        self.nb_actions = nb_actions
        assert self.nb_states.shape[0] == self.nb_interval
        # Create Actor and Critic Network
        net_cfg = {
            'hidden1':args.hidden1, 
            'hidden2':args.hidden2, 
            'init_w':args.init_w
        }

        self.actor_dict = dict()
        self.actor_target_dict = dict()
        self.actor_optim_dict = dict()
        self.actor_scheduler_dict = dict()


        self.critic_dict = dict()
        self.critic_target_dict = dict()
        self.critic_optim_dict = dict()
        self.critic_scheduler_dict = dict()

        self.memory_dict = dict()

        #build a model for each time interval
        for i in range(self.nb_interval):

            self.actor_dict[i] = Actor(self.nb_states[i],self.nb_actions, **net_cfg)
            self.actor_target_dict[i] = Actor(self.nb_states[i],self.nb_actions, **net_cfg)
            self.actor_optim_dict[i] = Adam(self.actor_dict[i].parameters(), lr=args.prate )
            self.actor_scheduler_dict[i] = ExponentialLR(self.actor_optim_dict[i],gamma = args.actorDecay)

            hard_update(self.actor_target_dict[i],self.actor_dict[i])
            
            self.critic_dict[i] = Critic(self.nb_states[i], self.nb_actions, **net_cfg)
            self.critic_target_dict[i] = Critic(self.nb_states[i],self.nb_actions,  **net_cfg)
            self.critic_optim_dict[i] = Adam(self.critic_dict[i].parameters(), lr=args.rate)
            self.critic_scheduler_dict[i] = ExponentialLR(self.critic_optim_dict[i],gamma = args.criticDecay)
            hard_update(self.critic_target_dict[i],self.critic_dict[i])



            
            #Create replay buffer
            self.memory_dict[i] = Memory(limit=args.rmsize)

        # Hyper-parameters
        self.batch_size = args.bsize
        self.tau = args.tau
        self.discount = args.discount

        # 
        self.s_t = None # Most recent state
        self.a_t = None # Most recent action
        self.is_training = True

        # 
        if USE_CUDA: self.cuda()

    def update_policy(self,time_interval):
        # Sample batch
        state_batch, action_batch, reward_batch, \
        next_state_batch = self.memory_dict[time_interval].sample_and_split(self.batch_size)

        # Prepare for the target q batch
        with torch.no_grad():
            if time_interval != self.nb_interval-1:
                next_q_values = self.critic_target_dict[time_interval+1](
                    to_tensor(next_state_batch),
                    self.actor_target_dict[time_interval+1](to_tensor(next_state_batch)),
                )

                target_q_batch = to_tensor(reward_batch) + \
                    self.discount*next_q_values
            else:
                target_q_batch = to_tensor(reward_batch)
        # Critic update
        self.critic_dict[time_interval].zero_grad()

        q_batch = self.critic_dict[time_interval]( to_tensor(state_batch), to_tensor(action_batch) )
        
        try:
            value_loss = criterion(torch.flatten(q_batch), torch.flatten(target_q_batch))
        except RuntimeError:
            print(torch.flatten(q_batch).shape)
            print(torch.flatten(target_q_batch).shape)
        try:
            value_loss.backward()
        except RuntimeError:
            print(q_batch)
            print(value_loss)
        self.critic_optim_dict[time_interval].step()

        # Actor update
        self.actor_dict[time_interval].zero_grad()

        policy_loss = -self.critic_dict[time_interval](
            to_tensor(state_batch),
            self.actor_dict[time_interval](to_tensor(state_batch))
        )


        policy_loss = policy_loss.mean()
        #policy_loss.requires_grad = True
        policy_loss.backward()
        self.actor_optim_dict[time_interval].step()

        # Target update
        soft_update(self.actor_target_dict[time_interval], self.actor_dict[time_interval], self.tau)
        soft_update(self.critic_target_dict[time_interval], self.critic_dict[time_interval], self.tau)
        self.actor_scheduler_dict[time_interval].step()
        self.critic_scheduler_dict[time_interval].step()

    def cuda(self):
        for i in range(self.nb_interval):
            self.actor_dict[i].cuda()
            self.actor_target_dict[i].cuda()
            self.critic_dict[i].cuda()
            self.critic_target_dict[i].cuda()

    def observe(self,s_t,a_t, r_t,s_t1,  time_interval):
        self.memory_dict[time_interval].append(s_t,a_t, r_t, s_t1)


    def random_action(self):
        action_cont = np.random.uniform(-1.,1.,size = (self.nb_actions,))
        return action_cont

    def select_action(self, s_t,time_interval):
        action_continuous = to_numpy(
            self.actor_dict[time_interval](to_tensor(np.array([s_t])))
        ).squeeze(0)


        return action_continuous

    def reset(self, obs):
        self.s_t = obs
        #self.random_process.reset_states()


    def seed(self,s):
        torch.manual_seed(s)
        if USE_CUDA:
            torch.cuda.manual_seed(s)
    def get_mc(self,s_t,action,time_interval):
      return self.critic_dict[time_interval].mc(to_tensor(np.array([s_t])),to_tensor(np.array([action])))

In [22]:
def rescale_action(action,lb,ub):
    return action * (ub- lb) / 2.0 +\
            (lb + ub) / 2.0
class Environment(object):
    def __init__(self, constraints,cost_func,elastic_load_func,load,impedance,nb_interval,gaussian_noise):
        self.constraints = constraints
        self.cost_func = cost_func
        self.elastic_load_func= elastic_load_func
        self.load = load
        self.impedance = impedance
        self.nb_interval = nb_interval
        self.gaussian_noise = gaussian_noise
        self.normal_scale = constraints['normal_scale']
        self.mc = np.array([0.]*nb_interval)
        self.trans = np.array([0.]*nb_interval)
        self.reset()
    def reset(self):
        self.battery_node1 = 0.0
        self.battery_node2 = 0.0
        self.observation_node1 =  np.array([self.battery_node1*self.constraints['battery_scale'],\
            (self.load[0]+self.trans[0])*self.constraints['load_scale']])
        self.observation_node2 = np.array([self.battery_node2*self.constraints['battery_scale'], self.mc[0]]) 
        self.state_node1 = np.array([self.battery_node1,\
            (self.load[0]+self.trans[0])])
        self.state_node2 = np.array([self.battery_node2,self.mc[0]])
        self.time_interval = 0
    def get_obs(self):
        return self.observation_node1,self.observation_node2,self.time_interval
    def reward(self,true_action,time_interval):
        if type(time_interval) != int:
            print(time_interval) 
        a_p,a_c1,a_c2,a_t,a_el,e_p,e_el,e_c1,e_c2 = true_action
        prod_cost = self.cost_func(e_p)

        #penalty for Node 1:
        penaltyForNode1 = -100*np.abs(a_p-e_p) - 100*np.abs(e_c1-a_c1)
        #penalty for Node 2:
        penaltyForNode2 = -100*np.abs(a_el-e_el)- 100*np.abs(e_c2-a_c2) - 1*np.abs(self.trans[time_interval]-a_t)
     
        
        el_profit = self.elastic_load_func(e_el)
        trans_cost = a_t*self.mc[time_interval]#negative
        return prod_cost , el_profit, trans_cost, penaltyForNode1,penaltyForNode2
    
    """
    the actions ouputed from Actors are in [-1,1], this function transfer them to actual values
    """   
    def make_action(self,action1, action2,t = None):
        if t is None:
            t = self.time_interval
        a_c1 = action1.item()#Charging of Node 1
        a_t,a_c2 = action2 # transmission and charging of Node 2
        load_t = self.load[t] + self.trans[t] #For node 1, the transmission work the same as fixed load.
        
        if self.gaussian_noise: #adding noise to actions for exploration

            a_c1 += np.random.normal(scale = self.normal_scale) 
            a_c1 = np.clip(a_c1,-1,1)
            
            a_c2 += np.random.normal(scale = self.normal_scale) 
            a_c2 = np.clip(a_c2,-1,1)
            
            a_t += np.random.normal(scale = self.normal_scale) 
            a_t = np.clip(a_t,-1,1)


        trans_t = rescale_action(a_t,0,self.constraints['max_angle']/self.impedance) #rescale it to true value (it is in [0,80])
        
        a_c1 = rescale_action(a_c1,-200,200)  #rescale it to true value ([-200,200])
        a_c2  = rescale_action(a_c2,-200,200) #rescale it to true value ([-200,200])
        
        
        lb_p = 0 #lower bound of production is 0
        ub_p = 600 #upper bound of procution is 600
        
        lb_el = 0 #lower bound of elastic load is 0
        ub_el = 150 #upper bound of elstic load is 150 (this can be adjusted)
        
        lb_c1 = -self.battery_node1 #lower bound of charging (or upper bound of discharging) of Node 1 is the energy in the battery
        ub_c1 = self.constraints['max_energy']-self.battery_node1 #upper bound of charging of Nope 2 is {battery capacity - current energy}

        lb_c2 = -self.battery_node2 #lower bound of charging (or upper bound of discharging) of Node 2 is the energy in the battery
        ub_c2 = self.constraints['max_energy']-self.battery_node2 #upper bound of charging of Nope 2 is {battery capacity - current energy}

        
        effective_c1 = np.minimum(np.maximum(a_c1, lb_c1),ub_c1) #compute the feasible charging decision
        effective_c2 = np.minimum(np.maximum(a_c2,lb_c2),ub_c2)
 
        a_p = effective_c1 + load_t #compute the production by power balance of Node 1
        a_el = trans_t - effective_c2 #compute the elastic load by power balance of Node 2

        effective_p =  np.minimum(np.maximum(a_p,lb_p),ub_p)  #compute the feasible production decision
        effective_el = np.maximum(a_el,lb_el)#compute the feasible elastic load decision 


        return np.array([a_p,a_c1,a_c2,trans_t,a_el,effective_p,effective_el,effective_c1,effective_c2])
        # return procution, charging of node 1, charging of node 2, transmission, elastic load, feasible proction, feasible elastic load, feasible charging 1 ,feasible charging 2

    def scale_decay(self): #decrease the noise variance (if applicable)
        self.normal_scale *= self.constraints['decay_rate']

    
    """
    update the marginal cost and transmission volume in the environment 
    """
    def updateMcAndTrans(self,new_mc,new_trans,updateRate,time_interval): 
        self.mc[time_interval] = (1-updateRate) * self.mc[time_interval] + updateRate*new_mc
        self.trans[time_interval] = (1-updateRate) * self.trans[time_interval] + updateRate*new_trans
    def step(self,action1,action2):
        action1_copy = deepcopy(action1)
        action2_copy = deepcopy(action2)
        true_action = self.make_action(action1,action2)
        r_1,r_2,r_t,pForN1,pForN2 = self.reward(true_action,self.time_interval)
        r_1 *= self.constraints['reward_scale']
        r_2 *= self.constraints['reward_scale']
        r_t *= self.constraints['reward_scale']#negative
        pForN1 *= self.constraints['reward_scale']
        pForN2 *= self.constraints['reward_scale']
        if self.time_interval == self.nb_interval -1:
            self.reset()
            return self.observation_node1,self.observation_node2,r_1,r_2,r_t,pForN1,pForN2,action1_copy,action2_copy,self.time_interval
        else:
            a_p,a_c1,a_c2,a_t,el,eP,eEl,eC1,eC2 = true_action
            self.battery_node1 += eC1
            self.battery_node2 += eC2
            self.time_interval += 1
            self.state_node1 =np.array([self.battery_node1,(self.load[self.time_interval]+self.trans[self.time_interval])\
              ])
            self.state_node2 = np.array([self.battery_node2,self.mc[self.time_interval]\
               ])
            self.observation_node1 = np.array([self.battery_node1*self.constraints['battery_scale'],(self.load[self.time_interval]+self.trans[self.time_interval])*self.constraints['load_scale']\
              ])
            self.observation_node2 = np.array([self.battery_node2*self.constraints['battery_scale'],self.mc[self.time_interval]\
               ])

            
            return self.observation_node1,self.observation_node2,r_1,r_2,r_t,pForN1,pForN2,action1_copy,action2_copy,self.time_interval

In [23]:
import logging
import sys
import argparse
import datetime

In [24]:
constraints = dict()
constraints['max_prod'] = 600
constraints['max_charge'] = 200
constraints['max_energy'] = 280
constraints['max_angle'] = 0.08
constraints['load_scale'] = 0.005
constraints['mc_scale'] = 0.05
constraints['battery_scale'] = 0.01
constraints['reward_scale'] = 0.0001
constraints['normal_scale'] = 0.05
constraints['decay_rate'] = 0.99

In [25]:
def cost_func(x):
  return -(20+2*x+0.02*x**2)
def elastic_load_func(x):
  return -0.02*x**2 + 12*x +5
def mc_func(x):
  return -0.04*x-2
 
impedance = 0.001
startup = 30
nb_interval = 8
nb_states_1 = np.array([1+1 for x in range(nb_interval)])
nb_states_2 = np.array([1+1 for x in range(nb_interval)])
nb_actions1 = 1
nb_actions2 = 2
load_1 = np.array([100,80,70,75,150,180,220,200]).astype(np.float32)
#load_1 = np.array([100,80,70]).astype(np.float32)
#load_1 = np.array([100])
#load_2 = np.array([120,100,80,80,170,200,230,200])
#load_2 = np.array([120,100,80,80,170,200,150,200])


In [26]:
parser = argparse.ArgumentParser(description='Power system DDPG')

parser.add_argument('--hidden1', default=128, type=int, help='hidden num of first fully connect layer')
parser.add_argument('--hidden2', default=256, type=int, help='hidden num of second fully connect layer')
parser.add_argument('--rate', default=0.01, type=float, help='learning rate')
parser.add_argument('--actorDecay', default=0.9995, type=float, help='gamma for actor optimization schedular')
parser.add_argument('--criticDecay', default=0.99995, type=float, help='gamma for critic optimization schedular')
parser.add_argument('--prate', default=0.001, type=float, help='policy net learning rate (only for DDPG)')
parser.add_argument('--warmup', default=5000, type=int, help='time without training but only filling the replay memory')
parser.add_argument('--discount', default=1., type=float, help='')
parser.add_argument('--bsize', default=256, type=int, help='minibatch size')
parser.add_argument('--rmsize', default=50000, type=int, help='memory size')
parser.add_argument('--tau', default=0.9, type=float, help='moving average for target network')
parser.add_argument('--init_w', default=0.03, type=float, help='') 
parser.add_argument('--train_iter', default=100000, type=int, help='train iters each timestep')
parser.add_argument('--seed', default=123, type=int, help='')
args = parser.parse_args("")



In [27]:
np.random.seed(123)

In [28]:

randomness = False #no gaussian noise if False
agent1 = DDPG(nb_interval,nb_states_1,nb_actions1, constraints, args)
agent2 = DDPG(nb_interval,nb_states_2,nb_actions2, constraints, args)
env = Environment(constraints,cost_func,elastic_load_func,load_1,impedance,nb_interval,randomness)

In [40]:
step = 0
episodeRewardNode1= 0.
episodeRewardNode2 = 0.
episodePenaltyNode1 = 0.
episodePenaltyNode2 = 0.
env.reset()
observation_1,observation_2,time_interval = env.get_obs()
trans_replay = []
trans_discrepancy = []
mc_rate = 0.5
while step < args.train_iter:


    if step < args.warmup:
        action1 = agent1.random_action()
        action2 = agent2.random_action()
    else:
      action1 = agent1.select_action(observation_1,time_interval)
      action2 = agent2.select_action(observation_2,time_interval)
      
      
    new_observation_1,new_observation_2,reward_1,reward_2,reward_t,penaltyForNode1,penaltyForNode2,action1,action2 ,new_time_interval = env.step(action1,action2)
    
    
    
    agent1.observe(observation_1,action1,reward_1+penaltyForNode1,new_observation_1,time_interval)
    agent2.observe(observation_2,action2,reward_2+reward_t + penaltyForNode2,new_observation_2,time_interval)

    true_action = env.make_action(deepcopy(action1),deepcopy(action2))
    a_p,a_c1,a_c2,a_t,el,eP,eEl,eC1,eC2 = true_action

    trans_discrepancy.append(np.abs(a_t-env.trans[time_interval]))
    new_mc = mc_func(a_p)
    new_trans = a_t
    env.updateMcAndTrans(new_mc,new_trans,0.5,time_interval)
    #update marginal cost
    #env.update_mc(mc_func(a_p),mc_rate*(step/100)**(-0.9),time_interval)
    
    if step > args.warmup and time_interval == 0:
      for k in range(env.nb_interval):
        agent1.update_policy(k)
        agent2.update_policy(k)


    observation_1 = new_observation_1
    observation_2 = new_observation_2
    time_interval = new_time_interval



    step += 1
    episodeRewardNode1 += reward_1 
    episodeRewardNode2 += reward_2# +reward_t

    episodePenaltyNode1 += penaltyForNode1
    episodePenaltyNode2 += penaltyForNode2
    #episode_reward += reward_1+reward_2 + penaltyForNode1+ penaltyForNode2
    if time_interval == 0 and display and step % 100 ==0:
        print("episode {} reward of Node 1 is {}\n".format(step,episodeRewardNode1)  )
        print("episode {} reward of Node 2 is {}\n".format(step,episodeRewardNode2))
        print("episode {} penalty of Node 1 is {}\n".format(step,episodePenaltyNode1) )
        print("episode {} penalty of Node 2 is {}\n".format(step,episodePenaltyNode2) )
        print("episode {} transmission discrepancy is".format(step),trans_discrepancy,"\n")

    if time_interval == 0:
        episodeRewardNode1 = 0.
        episodeRewardNode2 =0.
        episodePenaltyNode1 = 0.
        episodePenaltyNode2 = 0.
        trans_discrepancy = []
    if False and step > 3000:
        env.scale_decay()

episode 200 reward of Node 1 is -1.0987635230466115

episode 200 reward of Node 2 is 0.38677642397732537

episode 200 penalty of Node 1 is -2.624460518249437

episode 200 penalty of Node 2 is -2.5398270393118985

episode 200 transmission discrepancy is [19.114259513753055, 24.07247233802287, 17.55564257316705, 51.67849577087122, 25.541940232991568, 19.437017951824153, 15.865573293289081, 35.85385691112914] 

episode 400 reward of Node 1 is -1.1860757208876824

episode 400 reward of Node 2 is 0.35950720698256483

episode 400 penalty of Node 1 is -3.600244702857447

episode 400 penalty of Node 2 is -5.882760140632129

episode 400 transmission discrepancy is [33.706253778982585, 19.406809243992043, 59.82977011960531, 46.56859881794876, 4.963691622333169, 37.542628932370455, 31.00777229007554, 57.473030604736174] 

episode 600 reward of Node 1 is -1.2945243626880574

episode 600 reward of Node 2 is 0.43668329873583883

episode 600 penalty of Node 1 is -2.2560281768058648

episode 600 penal

In [42]:
env.mc

array([ -8.01032416,  -9.2399429 ,  -7.76521021,  -9.55300566,
       -10.25278069, -12.44073187, -11.05108533,  -8.46050024])

In [43]:
env.trans

array([61.51250259, 62.71230452, 74.21667482, 58.46730814, 33.76750102,
       30.14791235, 29.64798301, 32.12608197])

In [31]:
print('final raward {}'.format(-0.8518511254205375 + 0.42103376634055234))

final raward -0.4308173590799852
